In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import open3d as o3d
from pyntcloud import PyntCloud 
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN, OPTICS, SpectralCoclustering
from sklearn.metrics import pairwise_distances
import json
import os
from glob import glob
import random
import torch
np.random.seed(0)

In [2]:
##estimate statistics for normal surface
noise_std = 0
k_n = 50
sigma_list = []
data_folder = './Surface_Defects_pcd_extend_2000_estnorm_noise0001/'
filenames = json.load(open(data_folder + 'train_files.txt', 'r'))

for file in filenames:
    if file.split('/')[1] == 'normal':
        data = np.loadtxt(data_folder + file.split('/')[1] + '/' + file.split('/')[2], delimiter = ',')
        point_cloud = data[:,:3]
        point_cloud_noise = point_cloud + noise_std*np.random.randn(point_cloud.shape[0], point_cloud.shape[1])
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud_noise)
        pcd1 = PyntCloud.from_instance("open3d", pcd)

        # find neighbors
        kdtree_id = pcd1.add_structure("kdtree")
        k_neighbors = pcd1.get_neighbors(k=k_n, kdtree=kdtree_id) 

        # calculate eigenvalues
        ev = pcd1.add_scalar_field("eigen_values", k_neighbors=k_neighbors)

        x = pcd1.points['x'].values 
        y = pcd1.points['y'].values 
        z = pcd1.points['z'].values 

        e1 = pcd1.points['e3('+str(k_n+1)+')'].values
        e2 = pcd1.points['e2('+str(k_n+1)+')'].values
        e3 = pcd1.points['e1('+str(k_n+1)+')'].values

        sum_eg = np.add(np.add(e1,e2),e3)
        sigma = np.divide(e1,sum_eg)
        sigma_list.append(sigma)

sigma_arr = np.concatenate(sigma_list, axis = 0)            
normal_mean = sigma_arr.mean()
normal_std = sigma_arr.std()

In [3]:
cate_folder = glob("./Surface_Defects_pcd_extend_2000_estnorm_noise0001/*/")
root = "./Surface_Defects_pcd_extend_2000_estnorm_noise0001/"
files = []
folders = ['raised', 'dent', 'crack']
thresh = normal_mean + 1.282 * normal_std ##80% CI
for cate in cate_folder:
      
    filename = os.listdir(cate)

    for file in filename:

        if file.split('.')[-1] == 'txt':
            
            
            data = np.loadtxt(cate + file, delimiter = ',')
            point_cloud = data[:,:3]
            point_cloud_noise = point_cloud + noise_std*np.random.randn(point_cloud.shape[0], point_cloud.shape[1])
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(point_cloud_noise)

            pcd1 = PyntCloud.from_instance("open3d", pcd)

            # define hyperparameters
            
            # thresh = 0.0336
            # find neighbors
            kdtree_id = pcd1.add_structure("kdtree")
            k_neighbors = pcd1.get_neighbors(k=k_n, kdtree=kdtree_id) 

            # calculate eigenvalues
            ev = pcd1.add_scalar_field("eigen_values", k_neighbors=k_neighbors)

            x = pcd1.points['x'].values 
            y = pcd1.points['y'].values 
            z = pcd1.points['z'].values 

            e1 = pcd1.points['e3('+str(k_n+1)+')'].values
            e2 = pcd1.points['e2('+str(k_n+1)+')'].values
            e3 = pcd1.points['e1('+str(k_n+1)+')'].values

            sum_eg = np.add(np.add(e1,e2),e3)
            sigma = np.divide(e1,sum_eg)

            # visualize the edges
            sigma[sigma>thresh] = 1
            sigma[sigma<=thresh] = 0
            
            pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(10))
            pcd.orient_normals_consistent_tangent_plane(k=10)
            norms = np.asarray(pcd.normals)
            data[:,:3] = point_cloud_noise
            data[:,6:] = norms
            #fig = plt.figure()
            #ax = fig.add_subplot(111, projection='3d')
            np.savetxt(cate + file.split('.')[0] + '.txt', data, delimiter=',')
            if cate.split('/')[-2] in folders:
                np.savez(cate + file.split('.')[0] + '.npz', data = sigma)